In [ ]:
import struct
import sys
import serial
import binascii
import time
import numpy as np
import math

import os
import datetime
import matplotlib.pyplot as plt

# Local File Imports
from parse_bin_output import *

# specify the binary file path below

import os
import time

def detect_new_bin_file(path):
  output_dict = parse_ADC(path)
  return smooth_data(Car_finder(output_dict))

def monitor_directory(path):
  before = dict([(f, None) for f in os.listdir(path)])
  while True:
    time.sleep(1)
    after = dict([(f, None) for f in os.listdir(path)])
    added = [f for f in after if not f in before]
    if added:
      return detect_new_bin_file(path+"/"+added[0])
    before = after

if __name__ == "__main__":
  path = "../binData"
#     monitor_directory(path)